# 1.  Creating a chromatogram and fragment pickle file from an mzML

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path

In [4]:
import sys
sys.path.append('../..')

In [5]:
from vimms.DataGenerator import  get_data_source, get_spectral_feature_database
from vimms.Common import *

In [6]:
# set_log_level_info()
set_log_level_debug()

### Download beer and urine files

In [7]:
url = 'http://researchdata.gla.ac.uk/870/2/example_data.zip'
base_dir = os.path.join(os.getcwd(), 'example_data')

In [ ]:
if not os.path.isdir(base_dir): # if not exist then download the example data and extract it
    print('Creating %s' % base_dir)    
    out_file = 'example_data.zip'
    download_file(url, out_file)
    extract_zip_file(out_file, delete=True)
else:
    print('Found %s' % base_dir)

2020-07-30 17:16:51.977 | INFO     | vimms.Common:download_file:184 - Downloading example_data.zip
  0%|                                                                                                                                                                                                            | 0.00/869k [00:00<?, ?KB/s]

Creating C:\Users\Vinny\work\vimms\demo\01. Data\example_data


 84%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                               | 731k/869k [01:55<00:16, 8.32kKB/s]

### Generate Spectral Feature Database

In this section we demonstrate how ViMMS constructs the spectral feature database containing information, such as the densities of m/z, RT and intensities, scan durations, MS2 peaks, from the example Beer mzML files. The spectral feature database will be used to sample for various features during the simulation later.

The following two methods get_data_source and get_spectral_feature_database from ViMMS will be used.

- get_data_source loads a DataSource object that stores information on a set of .mzML files
- get_spectral_feature_database extracts relevant features from .mzML files that have been loaded into the DataSource.  

The parameter below should work for most cases, however for different data, it might be necessary to adjust the min_rt and max_rt values.

In [ ]:
filename = None                    # if None, use all mzML files found
min_ms1_intensity = 0              # min MS1 intensity threshold to include a data point for density estimation
min_ms2_intensity = 0              # min MS2 intensity threshold to include a data point for density estimation
min_rt = 0                         # min RT to include a data point for density estimation
max_rt = 1440                      # max RT to include a data point for density estimation
bandwidth_mz_intensity_rt = 1.0    # kernel bandwidth parameter to sample (mz, RT, intensity) values during simulation
bandwidth_n_peaks = 1.0            # kernel bandwidth parameter to sample number of peaks per scan during simulation

Load fullscan data and train spectral feature database

In [10]:
mzml_path = Path(base_dir, 'beers', 'fullscan', 'mzML')
xcms_output = Path(mzml_path, 'extracted_peaks_ms1.csv')
out_file = Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p')

In [11]:
ds_fullscan = get_data_source(mzml_path, filename, xcms_output)

2020-07-17 16:09:50.372 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_10_fullscan1.mzML
2020-07-17 16:09:52.516 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_11_fullscan1.mzML
2020-07-17 16:09:54.488 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_12_fullscan1.mzML
2020-07-17 16:09:56.233 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_13_fullscan1.mzML
2020-07-17 16:09:58.403 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_14_fullscan1.mzML
2020-07-17 16:10:00.782 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_15_fullscan1.mzML
2020-07-17 16:10:02.682 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_16_fullscan1.mzML
2020-07-17 16:10:04.613 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_17_fullscan1.mzML
2020-07-17 16:10:06.526 | INFO     | vimms.DataGenerator:load_data:161 - Loading

In [12]:
ps_fullscan = get_spectral_feature_database(ds_fullscan, filename, min_ms1_intensity, min_ms2_intensity, min_rt, max_rt,
               bandwidth_mz_intensity_rt, bandwidth_n_peaks, out_file)

2020-07-17 16:10:29.047 | DEBUG    | vimms.DataGenerator:__init__:436 - Extracted 0 MS2 scans
2020-07-17 16:10:29.048 | DEBUG    | vimms.DataGenerator:_compute_intensity_props:614 - Computing parent intensity proportions
2020-07-17 16:10:29.049 | DEBUG    | vimms.DataGenerator:__init__:445 - Extracting scan durations
2020-07-17 16:10:29.053 | DEBUG    | vimms.DataGenerator:_kde:626 - Training KDEs for ms_level=1
2020-07-17 16:10:29.054 | DEBUG    | vimms.DataGenerator:_kde:637 - Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x000002B4328C4358>
2020-07-17 16:10:29.056 | INFO     | vimms.DataGenerator:get_data:278 - Using values from XCMS peaklist
2020-07-17 16:10:29.221 | DEBUG    | vimms.DataGenerator:_kde:637 - Retrieving n_peaks values from <vimms.DataGenerator.DataSource object at 0x000002B4328C4358>
2020-07-17 16:11:49.955 | DEBUG    | vimms.DataGenerator:_kde:626 - Training KDEs for ms_level=2
2020-07-17 16:11:49.956 | DEBUG    | vimms.DataGenera

In [13]:
ps_fullscan.get_peak(1, 10) # try to sample 10 MS1 peaks

[Peak mz=195.6978 rt=543.16 intensity=27539.06 ms_level=1,
 Peak mz=397.1652 rt=262.21 intensity=1811009.10 ms_level=1,
 Peak mz=349.1197 rt=269.07 intensity=283477.74 ms_level=1,
 Peak mz=138.7118 rt=368.95 intensity=225290.59 ms_level=1,
 Peak mz=344.2195 rt=260.24 intensity=210378.74 ms_level=1,
 Peak mz=210.7901 rt=274.22 intensity=533981.65 ms_level=1,
 Peak mz=306.7243 rt=588.70 intensity=642613.48 ms_level=1,
 Peak mz=154.3001 rt=486.33 intensity=43865.63 ms_level=1,
 Peak mz=359.4897 rt=1084.65 intensity=28973.05 ms_level=1,
 Peak mz=240.2514 rt=550.73 intensity=3023915.09 ms_level=1]

Load fragmentation data and train spectral feature database

In [14]:
mzml_path = Path(base_dir, 'beers', 'fragmentation', 'mzML')
xcms_output = Path(mzml_path, 'extracted_peaks_ms1.csv')
out_file = Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fragmentation.p')

In [15]:
ds_fragmentation = get_data_source(mzml_path, filename, xcms_output)

2020-07-17 16:11:52.062 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_10_T10_POS.mzML
2020-07-17 16:12:04.154 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_11_T10_POS.mzML
2020-07-17 16:12:13.891 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_12_T10_POS.mzML
2020-07-17 16:12:24.664 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_13_T10_POS.mzML
2020-07-17 16:12:33.651 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_14_T10_POS.mzML
2020-07-17 16:12:45.341 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_15_T10_POS.mzML
2020-07-17 16:12:55.299 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_16_T10_POS.mzML
2020-07-17 16:13:06.628 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers_17_T10_POS.mzML
2020-07-17 16:13:17.637 | INFO     | vimms.DataGenerator:load_data:161 - Loading Beer_multibeers

In [16]:
ps = get_spectral_feature_database(ds_fragmentation, filename, min_ms1_intensity, min_ms2_intensity, min_rt, max_rt,
               bandwidth_mz_intensity_rt, bandwidth_n_peaks, out_file)

2020-07-17 16:15:12.411 | DEBUG    | vimms.DataGenerator:__init__:436 - Extracted 138969 MS2 scans
2020-07-17 16:15:12.412 | DEBUG    | vimms.DataGenerator:_compute_intensity_props:614 - Computing parent intensity proportions
2020-07-17 16:15:29.624 | DEBUG    | vimms.DataGenerator:__init__:445 - Extracting scan durations
2020-07-17 16:15:29.633 | DEBUG    | vimms.DataGenerator:_kde:626 - Training KDEs for ms_level=1
2020-07-17 16:15:29.633 | DEBUG    | vimms.DataGenerator:_kde:637 - Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x000002B45FADC550>
2020-07-17 16:15:29.634 | INFO     | vimms.DataGenerator:get_data:278 - Using values from XCMS peaklist
2020-07-17 16:15:29.758 | DEBUG    | vimms.DataGenerator:_kde:637 - Retrieving n_peaks values from <vimms.DataGenerator.DataSource object at 0x000002B45FADC550>
2020-07-17 16:19:26.910 | DEBUG    | vimms.DataGenerator:_kde:626 - Training KDEs for ms_level=2
2020-07-17 16:19:26.911 | DEBUG    | vimms.DataG

In [17]:
ps.get_peak(1, 10)

[Peak mz=161.2169 rt=982.16 intensity=42275.65 ms_level=1,
 Peak mz=436.1218 rt=735.86 intensity=934188.99 ms_level=1,
 Peak mz=225.4560 rt=420.52 intensity=54676.40 ms_level=1,
 Peak mz=428.4392 rt=991.06 intensity=48633.79 ms_level=1,
 Peak mz=353.6041 rt=1287.44 intensity=309046.30 ms_level=1,
 Peak mz=231.9866 rt=289.47 intensity=82752.74 ms_level=1,
 Peak mz=308.8029 rt=238.49 intensity=992237.53 ms_level=1,
 Peak mz=188.3706 rt=763.05 intensity=137722.74 ms_level=1,
 Peak mz=200.2215 rt=611.33 intensity=1026405.70 ms_level=1,
 Peak mz=274.3911 rt=256.36 intensity=250600.84 ms_level=1]

In [18]:
ps.get_peak(2, 10)

[Peak mz=100.7419 rt=422.10 intensity=13684.83 ms_level=2,
 Peak mz=52.1200 rt=412.18 intensity=5068.07 ms_level=2,
 Peak mz=85.3234 rt=981.91 intensity=1609.42 ms_level=2,
 Peak mz=59.6401 rt=1423.40 intensity=79.04 ms_level=2,
 Peak mz=85.9283 rt=1024.99 intensity=2687.45 ms_level=2,
 Peak mz=116.2092 rt=870.37 intensity=1818.78 ms_level=2,
 Peak mz=129.3453 rt=616.61 intensity=4722.44 ms_level=2,
 Peak mz=189.7386 rt=323.12 intensity=1166.00 ms_level=2,
 Peak mz=59.3254 rt=169.06 intensity=5590.01 ms_level=2,
 Peak mz=115.5208 rt=375.25 intensity=3432.25 ms_level=2]